# Imports

In [11]:

import pandas as pd
import os
import cv2
from feat import Detector
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from PIL import Image


# Dataset

In [12]:
file_path = './DiffusionFER/DiffusionEmotion_S/dataset_sheet.csv'  # Replace with the actual path to your dataset file

dataset = pd.read_csv(file_path)

dataset['subDirectory_filePath'] = dataset['subDirectory_filePath'].apply(lambda x: x.split('/')[-1])
dataset['subDirectory_filePath'] = dataset['subDirectory_filePath'].str.replace('.png', '', regex=False)

# Display the first few rows to verify structure
print(dataset.head())

  subDirectory_filePath  valence  arousal  expression
0            aksjlkjl_0     -0.1      0.1           0
1            aksndlkn_0      0.0      0.0           0
2            anavqmjd_0     -0.1     -0.1           0
3            aovjrrax_0     -0.2     -0.1           0
4            aptzlpuo_0     -0.1     -0.1           0


In [13]:
data_path = "./DiffusionFER/DiffusionEmotion_S/cropped/"

i = 1
image_paths = []
labels = []
image_files = []
for folder_name in os.listdir(data_path):
    i += 1
    if i == 6:
        break
    image_folder = data_path  + folder_name
    images = os.listdir(image_folder)
    for image in images:
        labels.append(folder_name)
        image_paths.append(image_folder + f'/{image}')
        image_files.append(cv2.imread(image_paths[-1]))

print(len(image_files), len(labels))


detector = Detector(device="cpu")


# List to store AU data for CSV
aus_data = []

data = pd.DataFrame(
    {'image_paths': image_paths,
     'labels': labels,
    })



INFO:root:Loading Face model: retinaface
C:\Users\Alina Lånedator\iis\Lib\site-packages\feat\face_detectors\Retinaface\Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimen

678 678


C:\Users\Alina Lånedator\iis\Lib\site-packages\feat\detector.py:224: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(
INFO:root:Loading facepose model:

# Calculate the accuracy of the model


In [14]:
import pickle
def train_and_eval(model, train_in, train_out, test_in, test_out):

    model.fit(train_in, train_out)
    predicted_val = model.predict(test_in)
    with open('model.pkl','wb') as f:
        pickle.dump(model,f)

    # Evaluate model
    f1 = f1_score(test_out, predicted_val, average='weighted')  # F1 score (weighted average for imbalanced classes)
    print(f'F1 Score (weighted) on test set: {f1:.2f}')
    print("\nClassification Report on test set:")
    print(classification_report(test_out, predicted_val))
    return accuracy_score(test_out, predicted_val)


def train_and_eval_regressor(model, train_in, train_out, test_in, test_out):
    model.fit(train_in, train_out)
    predicted_val = model.predict(test_in)
    # Evaluate model using regression metrics
    mse = mean_squared_error(test_out, predicted_val)
    r2 = r2_score(test_out, predicted_val)
    return mse, r2

# Preprocessing


In [15]:
au_file_path = './aus.csv'  # Replace with the actual path to your dataset file

au = pd.read_csv(au_file_path)
au['file'] = au['file'].apply(lambda x: x.split('/')[-1])
au.rename(columns={'file': 'subDirectory_filePath'}, inplace=True)

# merge the dataframe with the the au and emotion values
dataset = pd.merge(dataset, au, on='subDirectory_filePath', how='inner')

# Check for missing values in the dataset
print(dataset.isnull().sum())


dataset = dataset.dropna()

# Find rows with any NaN values
rows_with_nan = dataset[dataset.isnull().any(axis=1)]
print(rows_with_nan)

# Drop multiple columns
features = dataset.drop(columns=['valence', 'arousal', 'expression', 'face', 'subDirectory_filePath'])
labels = dataset['expression']



data_in, test_in, data_out, test_out = train_test_split(
features,
labels,
test_size=0.1,
random_state=42,
stratify=labels # balances labels across the sets
)
train_in, val_in, train_out, val_out = train_test_split(
data_in,
data_out,
test_size=(0.2/0.9), # 20% of the original data
random_state=42,
stratify=data_out
)

subDirectory_filePath    0
valence                  0
arousal                  0
expression               0
face                     0
AU01                     6
AU02                     6
AU04                     6
AU05                     6
AU06                     6
AU07                     6
AU09                     6
AU10                     6
AU11                     6
AU12                     6
AU14                     6
AU15                     6
AU17                     6
AU20                     6
AU23                     6
AU24                     6
AU25                     6
AU26                     6
AU28                     6
AU43                     6
dtype: int64
Empty DataFrame
Columns: [subDirectory_filePath, valence, arousal, expression, face, AU01, AU02, AU04, AU05, AU06, AU07, AU09, AU10, AU11, AU12, AU14, AU15, AU17, AU20, AU23, AU24, AU25, AU26, AU28, AU43]
Index: []

[0 rows x 25 columns]


# Train with Expression

In [16]:
# # Extract the target column
# labels = dataset['expression']
#
# train_in, test_in, train_out, test_out = train_test_split(
# features,
# labels,
# test_size=0.3,
# random_state=42,
# stratify=labels # balances labels across the sets
# )

# Train with Valence

In [17]:
# labels = dataset['valence']
#
# train_in, test_in, train_out, test_out = train_test_split(
# features,
# labels,
# test_size=0.3,
# random_state=42,
# stratify=labels # balances labels across the sets
# )

## Train with Random Forest

# Train

In [19]:

# Define the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 200, 500],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples at a leaf node
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False] , # Whether to bootstrap samples
    'criterion': ['gini', 'entropy']
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='f1_weighted',  # Optimize for F1 Score (weighted)
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available cores
)

print("Starting hyperparameter tuning...")
grid_search.fit(train_in, train_out)

# Best parameters and best model
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

best_model = grid_search.best_estimator_

# Evaluate on validation set
val_pred = best_model.predict(val_in)
val_f1 = f1_score(val_out, val_pred, average='weighted')
print(f'Validation F1 Score (weighted): {val_f1:.2f}')

# Evaluate on test set using train_and_eval function
print("Evaluating on test set...")
test_accuracy = train_and_eval(best_model, train_in, train_out, test_in, test_out)

Starting hyperparameter tuning...
Fitting 5 folds for each of 2160 candidates, totalling 10800 fits



KeyboardInterrupt



# XGBOOST

In [70]:
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Define the hyperparameter grid for XGBoost
xgb_param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [3, 6, 10],  # Depth of each tree
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate (step size)
    'subsample': [0.8, 1.0],  # Subsample ratio of the training set
    'colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns for each tree
    'gamma': [0, 1, 5]  # Minimum loss reduction to make a split
}

# Use GridSearchCV for XGBoost hyperparameter tuning
print("\nStarting XGBoost hyperparameter tuning...")
xgb_grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=xgb_param_grid,
    scoring='f1_weighted',  # Optimize for F1 Score (weighted)
    cv=5,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available cores
)
xgb_grid_search.fit(train_in, train_out)

# Best parameters for XGBoost
xgb_best_params = xgb_grid_search.best_params_
print(f"Best XGBoost Hyperparameters: {xgb_best_params}")

# Evaluate XGBoost on test set using train_and_eval
xgb_best_model = xgb_grid_search.best_estimator_
print("Evaluating XGBoost on test set...")
xgb_test_accuracy = train_and_eval(xgb_best_model, train_in, train_out, test_in, test_out)

# XGB Regressor


In [71]:
# xgb_regressor = xgb.XGBRegressor(
#     objective="reg:squarederror",
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=6,
#     random_state=42
# )
#
# # Train and evaluate the model
# mse, r2 = train_and_eval_regressor(xgb_regressor, train_in, train_out, test_in, test_out)
#
# # Print evaluation results
# print(f"Mean Squared Error (MSE): {mse}")
# print(f"R-squared (R²): {r2}")


Mean Squared Error (MSE): 0.02155014797127452
R-squared (R²): 0.8902573328853823


# Face and emotion detection with trained model

In [72]:
# import cv2
# import opencv_jupyter_ui as jcv2
# from feat import Detector
# from IPython.display import Image
#
# from feat.utils import FEAT_EMOTION_COLUMNS
#
# cam = cv2.VideoCapture(0)
# cam.set(cv2.CAP_PROP_BUFFERSIZE, 1)
#
# while True:
#     ret, frame = cam.read()
#     if not ret:
#         print("OpenCV found an error reading the next frame.")
#         break
#
#     faces = detector.detect_faces(frame)
#     landmarks = detector.detect_landmarks(frame, faces)
#     emotions = detector.detect_emotions(frame, faces, landmarks)
#
#     # The functions seem to assume a collection of images or frames. We acces "frame 0".
#     faces = faces[0]
#     landmarks = landmarks[0]
#     emotions = emotions[0]
#
#     strongest_emotion = emotions.argmax(axis=1)
#
#     for (face, top_emo) in zip(faces, strongest_emotion):
#         (x0, y0, x1, y1, p) = face
#         cv2.rectangle(frame, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 3)
#         cv2.putText(frame, FEAT_EMOTION_COLUMNS[top_emo], (int(x0), int(y0 - 10)), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
#
#     jcv2.imshow("Emotion Detection", frame)
#
#     key = jcv2.waitKey(1) & 0xFF
#     if key == 27: # ESC pressed
#         break
#
# cam.release()
# jcv2.destroyAllWindows()

In [73]:
# import cv2
# import os
# from feat import Detector
#
# # Initialize Py-Feat detector
# detector = Detector()
#
#
#
# def capture_and_process_image(cap, face_tracker, detector):
#     """
#     Captures an image from the webcam, saves it temporarily, and processes it using the detector.
#     """
#     # Read a frame from the webcam
#     ret, frame = cap.read()
#     if not ret:
#         print("Failed to capture frame.")
#         return None
#
#     # Convert to grayscale for face detection
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#
#     # Detect faces in the frame
#     faces = face_tracker.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
#     if len(faces) == 0:
#         print("No face detected.")
#         return None
#
#     # Process the first detected face (modify if you want to process multiple faces)
#     x, y, w, h = faces[0]
#     face_roi = frame[y:y + h, x:x + w]
#
#     # Save the face ROI as a temporary image
#     temp_image_path = "temp_face.jpg"
#     cv2.imwrite(temp_image_path, face_roi)
#
#     try:
#         # Pass the image path to the detector
#         result = detector.detect_image(temp_image_path)
#         # Predict the emotion using the Random Forest model
#         face_features = result.aus.values.flatten()
#         with open('model.pkl', 'rb') as f:
#             model = pickle.load(f)
#         emotion = model.predict([face_features])[0]  # Model expects 2D array
#         print(emotion)
#             # Display the predicted emotion
#         cv2.putText(frame, f'Emotion: {emotion}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#
#         # Delete the temporary image
#         # if os.path.exists(temp_image_path):
#         #     os.remove(temp_image_path)
#
#         return result
#     except Exception as e:
#         print(f"Error during detection: {e}")
#         # Clean up the temporary file in case of error
#         if os.path.exists(temp_image_path):
#             os.remove(temp_image_path)
#         return None
#
#
# def main():
#     face_tracker = cv2.CascadeClassifier("frontal_face_features.xml")
#
#     # Open the laptop webcam (default device is 0)
#     cap = cv2.VideoCapture(0)
#
#     # Check if the webcam is opened correctly
#     if not cap.isOpened():
#         print("Error: Could not access the webcam.")
#         exit()
#
#     print("Press 'Enter' to capture and process an image.")
#     print("Press 'ESC' to exit.")
#
#     while True:
#         # Capture frame-by-frame
#         ret, frame = cap.read()
#
#         # If the frame was not captured correctly, break the loop
#         if not ret:
#             print("Failed to grab frame.")
#             break
#
#         # Display the live feed
#         cv2.imshow("Live Feed", frame)
#
#         # Check for key presses
#         key = cv2.waitKey(1) & 0xFF
#         if key == 27:  # Press 'ESC' to exit the program
#             break
#         elif key == 13:  # Press 'Enter' to capture and process an image
#             detection_result = capture_and_process_image(cap, face_tracker, detector)
#
#
#
#     # Release the capture and close the OpenCV window
#     cap.release()
#     cv2.destroyAllWindows()
#
#
# if __name__ == "__main__":
#     main()


In [74]:
# # Function to load and preprocess images
# def load_and_preprocess_images(image_paths, image_size=(64, 64)):
#     """
#     Loads and preprocesses images from the provided paths.
#     Resizes images to the given image_size and flattens them into 1D arrays.
#     """
#     images = []
#     for image_path in image_paths:
#         # Open the image file
#         img = Image.open(image_path)
#         # Resize image to a fixed size
#         img = img.resize(image_size)
#         # Convert image to numpy array and flatten it
#         img_array = np.array(img).flatten()
#         images.append(img_array)
#
#     return np.array(images)
#
# # Assuming 'dataset' is your DataFrame containing image paths and labels
# # Example dataset structure
# data = pd.DataFrame({
#     'image_paths': data["image_paths"],  # List of image paths
#     'labels': data["labels"]  # Corresponding labels
# })
# for image_path in data['image_paths']:
#
#     result = detector.detect_image(image_path)
#     num_rows = len(result)
#     new_df = result.aus
#     image_path = image_path.replace(".png", "")
#     image_path = image_path.replace("./DiffusionFER/DiffusionEmotion_S/cropped/", "")
#     new_df.insert(0, "file", image_path)
#     new_df.insert(1, "face", range(num_rows))
#
#     aus_data.append(new_df)
#
#     aus_data.append(new_df)
#
#
# # Merge all AU data and save as a CSV
# merged_df = pd.concat(aus_data, ignore_index=True)
# aus_df = pd.DataFrame(merged_df)
# aus_df.to_csv("aus.csv", index=False)
# # Load and preprocess images
# images = load_and_preprocess_images(data['image_paths'].values)
#
# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(images, data['labels'], test_size=0.2, random_state=42, stratify=data['labels'])
#
# # Train Random Forest model
# rf_model = RandomForestClassifier(n_estimators=150, random_state=42, class_weight='balanced')
# rf_model.fit(X_train, y_train)
#
# # Evaluate the model
# y_pred = rf_model.predict(X_test)
#
# # Print evaluation results
# # print("Accuracy on Test Set:", accuracy_score(y_test, y_pred))
# # print("Classification Report:\n", classification_report(y_test, y_pred))
# # print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
